# Exercise 8.4A: Comprehensive Feature Extraction Pipeline for DGA Detection

**Type**: Hands-on Implementation  
**Duration**: 3-4 hours  
**Difficulty**: Intermediate

---

## 📋 Learning Objectives

By completing this notebook, you will:
- ✅ Understand Domain Generation Algorithm (DGA) detection principles
- ✅ Implement comprehensive feature extraction for domains
- ✅ Build production-grade, scalable feature pipelines
- ✅ Validate feature quality and discriminative power
- ✅ Optimize code for real-time performance (<1ms per domain)

---

## 🎯 The Problem: Why DGA Detection Matters

**Domain Generation Algorithms (DGAs)** are used by malware to:
- Generate thousands of domain names dynamically
- Evade blacklists and static domain blocking
- Establish command-and-control (C2) communications
- Rotate domains to avoid detection

**Our Mission**: Build a feature extraction system that can process 100,000+ domains per hour in a Security Operations Center (SOC) environment.

---

## 📦 Prerequisites & Setup

Before starting, ensure you have:
- Intermediate Python programming skills
- Understanding of feature engineering concepts
- Familiarity with NumPy and Pandas

In [1]:
# Install required libraries (run this cell first if packages are missing)
# Uncomment the line below if needed:
# !pip install pandas numpy scipy scikit-learn matplotlib seaborn

import pandas as pd
import numpy as np
import re
import math
from collections import Counter
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import time
import warnings

warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

✅ All libraries imported successfully!
Pandas version: 2.3.3
NumPy version: 2.4.0


---

## 🔬 Part 1: Understanding Domain Features

Before implementing our pipeline, let's understand what makes a domain "suspicious."

### Feature Categories

1. **Lexical Features**: Character-level patterns
   - Length, digit ratio, entropy, special characters
   
2. **Linguistic Features**: Language-like properties
   - Pronounceability, vowel-consonant patterns, n-grams
   
3. **Statistical Features**: Distribution analysis
   - Character frequency, bigram entropy, randomness
   
4. **DNS Features**: Domain structure
   - TLD classification, subdomain count, label patterns

Let's examine some examples:

In [2]:
# Example domains for analysis
example_domains = {
    'legitimate': ['google.com', 'facebook.com', 'amazon.co.uk', 'stackoverflow.com'],
    'dga_cryptolocker': ['acmipywpotq.net', 'bdfkqrxtyuc.com', 'ceglosuvwxz.org'],
    'dga_conficker': ['jkxpvqwdza.biz', 'nmsuyzbfhj.info', 'plqtwxzach.ru']
}

print("=" * 60)
print("DOMAIN EXAMPLES")
print("=" * 60)

for category, domains in example_domains.items():
    print(f"\n{category.upper().replace('_', ' ')}:")
    for domain in domains:
        print(f"  • {domain}")
        
print("\n" + "=" * 60)
print("Notice the differences:")
print("  - Legitimate: readable, meaningful words")
print("  - DGA: random-looking, high entropy, less pronounceable")
print("=" * 60)

DOMAIN EXAMPLES

LEGITIMATE:
  • google.com
  • facebook.com
  • amazon.co.uk
  • stackoverflow.com

DGA CRYPTOLOCKER:
  • acmipywpotq.net
  • bdfkqrxtyuc.com
  • ceglosuvwxz.org

DGA CONFICKER:
  • jkxpvqwdza.biz
  • nmsuyzbfhj.info
  • plqtwxzach.ru

Notice the differences:
  - Legitimate: readable, meaningful words
  - DGA: random-looking, high entropy, less pronounceable


---

## 🛠️ Task 1: Implement Feature Extraction Modules

We'll build four modular feature extractors. Each focuses on a specific aspect of domain analysis.

### 1.1 Lexical Feature Extractor

Lexical features capture character-level patterns and structural properties.

In [3]:
class LexicalFeatureExtractor:
    """
    Extracts lexical (character-level) features from domain names.
    
    Features:
    - Length metrics
    - Character composition (digits, vowels, consonants)
    - Entropy (randomness measure)
    - Special character patterns
    """
    
    def __init__(self):
        self.vowels = set('aeiou')
        self.consonants = set('bcdfghjklmnpqrstvwxyz')
        
    def extract(self, domain: str) -> Dict[str, float]:
        """
        Extract all lexical features from a domain.
        
        Args:
            domain: Domain name (e.g., 'google.com')
            
        Returns:
            Dictionary of feature name -> value
        """
        # Extract domain name without TLD for analysis
        domain_parts = domain.split('.')
        domain_name = domain_parts[0].lower()
        
        features = {}
        
        # 1. Length Features
        features['length'] = len(domain_name)
        features['length_total'] = len(domain)
        
        # 2. Character Composition
        features['digit_count'] = sum(c.isdigit() for c in domain_name)
        features['digit_ratio'] = features['digit_count'] / max(len(domain_name), 1)
        
        features['vowel_count'] = sum(c in self.vowels for c in domain_name)
        features['vowel_ratio'] = features['vowel_count'] / max(len(domain_name), 1)
        
        features['consonant_count'] = sum(c in self.consonants for c in domain_name)
        features['consonant_ratio'] = features['consonant_count'] / max(len(domain_name), 1)
        
        # 3. Entropy (Shannon Entropy)
        features['entropy'] = self._calculate_entropy(domain_name)
        
        # 4. Special Characters
        features['hyphen_count'] = domain_name.count('-')
        features['underscore_count'] = domain_name.count('_')
        features['special_char_ratio'] = (features['hyphen_count'] + 
                                         features['underscore_count']) / max(len(domain_name), 1)
        
        # 5. Case Patterns (for full domain)
        features['uppercase_count'] = sum(c.isupper() for c in domain)
        features['uppercase_ratio'] = features['uppercase_count'] / max(len(domain), 1)
        
        # 6. Consecutive Character Patterns
        features['max_consecutive_digits'] = self._max_consecutive_type(domain_name, str.isdigit)
        features['max_consecutive_consonants'] = self._max_consecutive_chars(domain_name, self.consonants)
        
        return features
    
    def _calculate_entropy(self, text: str) -> float:
        """
        Calculate Shannon entropy.
        
        Entropy measures randomness:
        - Low entropy: Predictable (e.g., 'aaaa' or 'google')
        - High entropy: Random (e.g., 'xqzpkwjt')
        
        Formula: H(X) = -Σ p(x) * log2(p(x))
        """
        if not text:
            return 0.0
        
        # Count character frequencies
        char_counts = Counter(text)
        length = len(text)
        
        # Calculate probability and entropy
        entropy = 0.0
        for count in char_counts.values():
            probability = count / length
            entropy -= probability * math.log2(probability)
        
        return entropy
    
    def _max_consecutive_type(self, text: str, check_func) -> int:
        """Find maximum consecutive characters matching a condition."""
        if not text:
            return 0
        
        max_count = 0
        current_count = 0
        
        for char in text:
            if check_func(char):
                current_count += 1
                max_count = max(max_count, current_count)
            else:
                current_count = 0
        
        return max_count
    
    def _max_consecutive_chars(self, text: str, char_set: set) -> int:
        """Find maximum consecutive characters from a set."""
        if not text:
            return 0
        
        max_count = 0
        current_count = 0
        
        for char in text:
            if char in char_set:
                current_count += 1
                max_count = max(max_count, current_count)
            else:
                current_count = 0
        
        return max_count

# Test the lexical extractor
print("🧪 Testing Lexical Feature Extractor\n")

lexical_extractor = LexicalFeatureExtractor()

test_domains = ['google.com', 'xqzpkwjt.net', 'amazon123.com']

for domain in test_domains:
    features = lexical_extractor.extract(domain)
    print(f"Domain: {domain}")
    print(f"  Length: {features['length']}")
    print(f"  Entropy: {features['entropy']:.3f}")
    print(f"  Vowel Ratio: {features['vowel_ratio']:.3f}")
    print(f"  Digit Ratio: {features['digit_ratio']:.3f}")
    print()

print("✅ Lexical extractor working!")

🧪 Testing Lexical Feature Extractor

Domain: google.com
  Length: 6
  Entropy: 1.918
  Vowel Ratio: 0.500
  Digit Ratio: 0.000

Domain: xqzpkwjt.net
  Length: 8
  Entropy: 3.000
  Vowel Ratio: 0.000
  Digit Ratio: 0.000

Domain: amazon123.com
  Length: 9
  Entropy: 2.948
  Vowel Ratio: 0.333
  Digit Ratio: 0.333

✅ Lexical extractor working!


### 1.2 Linguistic Feature Extractor

Linguistic features capture language-like properties. DGA domains often lack pronounceability.

In [4]:
class LinguisticFeatureExtractor:
    """
    Extracts linguistic features - how 'language-like' is the domain?
    
    Features:
    - Pronounceability score
    - Vowel-consonant patterns
    - N-gram analysis (bigrams, trigrams)
    - Dictionary word presence
    """
    
    def __init__(self):
        self.vowels = set('aeiou')
        
        # Common English bigrams (top frequency)
        self.common_bigrams = {
            'th', 'he', 'in', 'er', 'an', 're', 'on', 'at', 'en', 'nd',
            'ti', 'es', 'or', 'te', 'of', 'ed', 'is', 'it', 'al', 'ar'
        }
        
        # Common English trigrams
        self.common_trigrams = {
            'the', 'and', 'ing', 'ion', 'tio', 'ent', 'ati', 'for', 'her', 'ter'
        }
    
    def extract(self, domain: str) -> Dict[str, float]:
        """Extract linguistic features."""
        domain_name = domain.split('.')[0].lower()
        
        features = {}
        
        # 1. Pronounceability Score
        features['pronounceability'] = self._calculate_pronounceability(domain_name)
        
        # 2. Vowel-Consonant Transitions
        features['vowel_consonant_transitions'] = self._count_vc_transitions(domain_name)
        features['vc_transition_ratio'] = features['vowel_consonant_transitions'] / max(len(domain_name) - 1, 1)
        
        # 3. N-gram Analysis
        bigrams = self._extract_ngrams(domain_name, 2)
        trigrams = self._extract_ngrams(domain_name, 3)
        
        features['common_bigram_count'] = sum(1 for bg in bigrams if bg in self.common_bigrams)
        features['common_bigram_ratio'] = features['common_bigram_count'] / max(len(bigrams), 1)
        
        features['common_trigram_count'] = sum(1 for tg in trigrams if tg in self.common_trigrams)
        features['common_trigram_ratio'] = features['common_trigram_count'] / max(len(trigrams), 1)
        
        # 4. Unique N-grams (diversity)
        features['unique_bigram_ratio'] = len(set(bigrams)) / max(len(bigrams), 1)
        features['unique_trigram_ratio'] = len(set(trigrams)) / max(len(trigrams), 1)
        
        # 5. Repeating Patterns
        features['repeating_patterns'] = self._find_repeating_patterns(domain_name)
        
        return features
    
    def _calculate_pronounceability(self, text: str) -> float:
        """
        Calculate pronounceability score (0-1).
        
        Based on vowel-consonant alternation. Languages typically alternate
        between vowels and consonants (e.g., 'banana', 'google').
        Random strings have poor alternation (e.g., 'xqzpk').
        """
        if len(text) < 2:
            return 0.5
        
        # Count good transitions (V->C or C->V)
        good_transitions = 0
        
        for i in range(len(text) - 1):
            curr_is_vowel = text[i] in self.vowels
            next_is_vowel = text[i + 1] in self.vowels
            
            # Good if alternating
            if curr_is_vowel != next_is_vowel:
                good_transitions += 1
        
        return good_transitions / (len(text) - 1)
    
    def _count_vc_transitions(self, text: str) -> int:
        """Count vowel-consonant transitions."""
        if len(text) < 2:
            return 0
        
        transitions = 0
        for i in range(len(text) - 1):
            curr_is_vowel = text[i] in self.vowels
            next_is_vowel = text[i + 1] in self.vowels
            
            if curr_is_vowel != next_is_vowel:
                transitions += 1
        
        return transitions
    
    def _extract_ngrams(self, text: str, n: int) -> List[str]:
        """Extract n-grams from text."""
        if len(text) < n:
            return []
        
        return [text[i:i+n] for i in range(len(text) - n + 1)]
    
    def _find_repeating_patterns(self, text: str) -> int:
        """Count repeating character patterns."""
        if len(text) < 2:
            return 0
        
        repeats = 0
        for i in range(len(text) - 1):
            if text[i] == text[i + 1]:
                repeats += 1
        
        return repeats

# Test the linguistic extractor
print("🧪 Testing Linguistic Feature Extractor\n")

linguistic_extractor = LinguisticFeatureExtractor()

test_domains = ['google.com', 'xqzpkwjt.net', 'banana.com']

for domain in test_domains:
    features = linguistic_extractor.extract(domain)
    print(f"Domain: {domain}")
    print(f"  Pronounceability: {features['pronounceability']:.3f}")
    print(f"  VC Transitions: {features['vowel_consonant_transitions']}")
    print(f"  Common Bigrams: {features['common_bigram_count']}")
    print(f"  Common Trigrams: {features['common_trigram_count']}")
    print()

print("✅ Linguistic extractor working!")

🧪 Testing Linguistic Feature Extractor

Domain: google.com
  Pronounceability: 0.600
  VC Transitions: 3
  Common Bigrams: 0
  Common Trigrams: 0

Domain: xqzpkwjt.net
  Pronounceability: 0.000
  VC Transitions: 0
  Common Bigrams: 0
  Common Trigrams: 0

Domain: banana.com
  Pronounceability: 1.000
  VC Transitions: 5
  Common Bigrams: 2
  Common Trigrams: 0

✅ Linguistic extractor working!


### 1.3 Statistical Feature Extractor

Statistical features analyze the distribution of characters and patterns.

In [5]:
class StatisticalFeatureExtractor:
    """
    Extracts statistical features based on character distributions.
    
    Features:
    - Character frequency variance
    - N-gram entropy
    - Randomness tests
    - Distribution metrics
    """
    
    def extract(self, domain: str) -> Dict[str, float]:
        """Extract statistical features."""
        domain_name = domain.split('.')[0].lower()
        
        features = {}
        
        # 1. Character Frequency Analysis
        char_freq = Counter(domain_name)
        frequencies = list(char_freq.values())
        
        if frequencies:
            features['char_freq_mean'] = np.mean(frequencies)
            features['char_freq_std'] = np.std(frequencies)
            features['char_freq_variance'] = np.var(frequencies)
            features['unique_char_ratio'] = len(char_freq) / max(len(domain_name), 1)
        else:
            features['char_freq_mean'] = 0
            features['char_freq_std'] = 0
            features['char_freq_variance'] = 0
            features['unique_char_ratio'] = 0
        
        # 2. N-gram Entropy
        bigrams = self._extract_ngrams(domain_name, 2)
        features['bigram_entropy'] = self._calculate_ngram_entropy(bigrams)
        
        trigrams = self._extract_ngrams(domain_name, 3)
        features['trigram_entropy'] = self._calculate_ngram_entropy(trigrams)
        
        # 3. Character Distribution Skewness & Kurtosis
        if len(frequencies) > 1:
            features['char_freq_skewness'] = stats.skew(frequencies)
            features['char_freq_kurtosis'] = stats.kurtosis(frequencies)
        else:
            features['char_freq_skewness'] = 0
            features['char_freq_kurtosis'] = 0
        
        # 4. Randomness Score (combining multiple metrics)
        features['randomness_score'] = self._calculate_randomness(domain_name)
        
        # 5. Alphabetic Position Statistics
        features['avg_char_position'] = self._avg_alphabetic_position(domain_name)
        features['char_position_std'] = self._std_alphabetic_position(domain_name)
        
        return features
    
    def _extract_ngrams(self, text: str, n: int) -> List[str]:
        """Extract n-grams."""
        if len(text) < n:
            return []
        return [text[i:i+n] for i in range(len(text) - n + 1)]
    
    def _calculate_ngram_entropy(self, ngrams: List[str]) -> float:
        """Calculate entropy of n-gram distribution."""
        if not ngrams:
            return 0.0
        
        ngram_counts = Counter(ngrams)
        total = len(ngrams)
        
        entropy = 0.0
        for count in ngram_counts.values():
            probability = count / total
            entropy -= probability * math.log2(probability)
        
        return entropy
    
    def _calculate_randomness(self, text: str) -> float:
        """
        Calculate overall randomness score (0-1).
        
        Combines:
        - High entropy
        - High unique character ratio
        - Low pronounceability
        """
        if not text:
            return 0.0
        
        # Entropy component (normalized)
        char_counts = Counter(text)
        entropy = 0.0
        for count in char_counts.values():
            p = count / len(text)
            entropy -= p * math.log2(p)
        
        max_entropy = math.log2(len(text)) if len(text) > 1 else 1
        normalized_entropy = entropy / max_entropy if max_entropy > 0 else 0
        
        # Unique character component
        uniqueness = len(char_counts) / len(text)
        
        # Combine
        randomness = (normalized_entropy + uniqueness) / 2
        
        return randomness
    
    def _avg_alphabetic_position(self, text: str) -> float:
        """Average position of characters in alphabet (a=1, z=26)."""
        positions = []
        for char in text:
            if char.isalpha():
                positions.append(ord(char.lower()) - ord('a') + 1)
        
        return np.mean(positions) if positions else 0.0
    
    def _std_alphabetic_position(self, text: str) -> float:
        """Standard deviation of character positions in alphabet."""
        positions = []
        for char in text:
            if char.isalpha():
                positions.append(ord(char.lower()) - ord('a') + 1)
        
        return np.std(positions) if positions else 0.0

# Test the statistical extractor
print("🧪 Testing Statistical Feature Extractor\n")

statistical_extractor = StatisticalFeatureExtractor()

test_domains = ['google.com', 'xqzpkwjt.net', 'aaabbbccc.com']

for domain in test_domains:
    features = statistical_extractor.extract(domain)
    print(f"Domain: {domain}")
    print(f"  Unique Char Ratio: {features['unique_char_ratio']:.3f}")
    print(f"  Bigram Entropy: {features['bigram_entropy']:.3f}")
    print(f"  Randomness Score: {features['randomness_score']:.3f}")
    print(f"  Char Freq Variance: {features['char_freq_variance']:.3f}")
    print()

print("✅ Statistical extractor working!")

🧪 Testing Statistical Feature Extractor

Domain: google.com
  Unique Char Ratio: 0.667
  Bigram Entropy: 2.322
  Randomness Score: 0.704
  Char Freq Variance: 0.250

Domain: xqzpkwjt.net
  Unique Char Ratio: 1.000
  Bigram Entropy: 2.807
  Randomness Score: 1.000
  Char Freq Variance: 0.000

Domain: aaabbbccc.com
  Unique Char Ratio: 0.333
  Bigram Entropy: 2.250
  Randomness Score: 0.417
  Char Freq Variance: 0.000

✅ Statistical extractor working!


### 1.4 DNS Feature Extractor

DNS features capture domain structure and naming conventions.

In [6]:
class DNSFeatureExtractor:
    """
    Extracts DNS-related structural features.
    
    Features:
    - TLD classification
    - Subdomain analysis
    - Label patterns
    - Domain structure
    """
    
    def __init__(self):
        # Common legitimate TLDs
        self.common_tlds = {
            'com', 'org', 'net', 'edu', 'gov', 'co', 'io', 'info', 'biz'
        }
        
        # Suspicious TLDs (often used in malware campaigns)
        self.suspicious_tlds = {
            'tk', 'ml', 'ga', 'cf', 'gq', 'pw', 'cc', 'xyz', 'top'
        }
    
    def extract(self, domain: str) -> Dict[str, float]:
        """Extract DNS structure features."""
        parts = domain.split('.')
        
        features = {}
        
        # 1. TLD Analysis
        if len(parts) >= 2:
            tld = parts[-1].lower()
            features['tld_is_common'] = 1 if tld in self.common_tlds else 0
            features['tld_is_suspicious'] = 1 if tld in self.suspicious_tlds else 0
            features['tld_length'] = len(tld)
        else:
            features['tld_is_common'] = 0
            features['tld_is_suspicious'] = 0
            features['tld_length'] = 0
        
        # 2. Subdomain Analysis
        features['subdomain_count'] = len(parts) - 2 if len(parts) > 2 else 0
        features['has_subdomain'] = 1 if features['subdomain_count'] > 0 else 0
        
        # 3. Label Analysis
        features['total_labels'] = len(parts)
        
        if parts:
            label_lengths = [len(p) for p in parts]
            features['avg_label_length'] = np.mean(label_lengths)
            features['max_label_length'] = max(label_lengths)
            features['min_label_length'] = min(label_lengths)
        else:
            features['avg_label_length'] = 0
            features['max_label_length'] = 0
            features['min_label_length'] = 0
        
        # 4. Domain Name (without TLD) Analysis
        if parts:
            domain_name = parts[0]
            features['domain_name_length'] = len(domain_name)
            
            # Check for numerical suffix (common in DGA)
            features['has_numerical_suffix'] = 1 if domain_name and domain_name[-1].isdigit() else 0
        else:
            features['domain_name_length'] = 0
            features['has_numerical_suffix'] = 0
        
        # 5. Dot Density
        features['dot_count'] = domain.count('.')
        features['dot_density'] = domain.count('.') / max(len(domain), 1)
        
        return features

# Test the DNS extractor
print("🧪 Testing DNS Feature Extractor\n")

dns_extractor = DNSFeatureExtractor()

test_domains = ['google.com', 'mail.google.com', 'xqzpk123.tk', 'amazon.co.uk']

for domain in test_domains:
    features = dns_extractor.extract(domain)
    print(f"Domain: {domain}")
    print(f"  TLD Common: {features['tld_is_common']}")
    print(f"  TLD Suspicious: {features['tld_is_suspicious']}")
    print(f"  Subdomain Count: {features['subdomain_count']}")
    print(f"  Total Labels: {features['total_labels']}")
    print()

print("✅ DNS extractor working!")

🧪 Testing DNS Feature Extractor

Domain: google.com
  TLD Common: 1
  TLD Suspicious: 0
  Subdomain Count: 0
  Total Labels: 2

Domain: mail.google.com
  TLD Common: 1
  TLD Suspicious: 0
  Subdomain Count: 1
  Total Labels: 3

Domain: xqzpk123.tk
  TLD Common: 0
  TLD Suspicious: 1
  Subdomain Count: 0
  Total Labels: 2

Domain: amazon.co.uk
  TLD Common: 0
  TLD Suspicious: 0
  Subdomain Count: 1
  Total Labels: 3

✅ DNS extractor working!


---

## 🏗️ Task 2: Build Scalable Pipeline

Now let's combine all extractors into a unified, production-ready pipeline.

In [7]:
class DomainFeatureExtractor:
    """
    Unified feature extraction pipeline for domain names.
    
    Combines all feature modules:
    - Lexical
    - Linguistic
    - Statistical
    - DNS
    
    Optimized for:
    - Batch processing
    - Performance (>1000 domains/second)
    - Error handling
    - Feature scaling
    """
    
    def __init__(self, enable_scaling: bool = False):
        """
        Initialize the feature extractor.
        
        Args:
            enable_scaling: Whether to apply feature scaling
        """
        self.lexical_extractor = LexicalFeatureExtractor()
        self.linguistic_extractor = LinguisticFeatureExtractor()
        self.statistical_extractor = StatisticalFeatureExtractor()
        self.dns_extractor = DNSFeatureExtractor()
        
        self.enable_scaling = enable_scaling
        self.scaler = StandardScaler() if enable_scaling else None
        self.feature_names = []
        self._is_fitted = False
    
    def extract_single(self, domain: str) -> Dict[str, float]:
        """
        Extract all features from a single domain.
        
        Args:
            domain: Domain name
            
        Returns:
            Dictionary of all features
        """
        try:
            features = {}
            
            # Extract from all modules
            features.update(self.lexical_extractor.extract(domain))
            features.update(self.linguistic_extractor.extract(domain))
            features.update(self.statistical_extractor.extract(domain))
            features.update(self.dns_extractor.extract(domain))
            
            return features
            
        except Exception as e:
            print(f"⚠️ Error processing domain '{domain}': {str(e)}")
            return {}
    
    def extract_batch(self, domains: List[str], show_progress: bool = True) -> pd.DataFrame:
        """
        Extract features from a batch of domains.
        
        Args:
            domains: List of domain names
            show_progress: Whether to show progress
            
        Returns:
            DataFrame with features for all domains
        """
        features_list = []
        
        start_time = time.time()
        
        for i, domain in enumerate(domains):
            features = self.extract_single(domain)
            if features:
                features['domain'] = domain
                features_list.append(features)
            
            # Progress indicator
            if show_progress and (i + 1) % 1000 == 0:
                elapsed = time.time() - start_time
                rate = (i + 1) / elapsed
                print(f"Processed {i + 1}/{len(domains)} domains ({rate:.0f} domains/sec)")
        
        df = pd.DataFrame(features_list)
        
        # Move domain column to first position
        if 'domain' in df.columns:
            cols = ['domain'] + [col for col in df.columns if col != 'domain']
            df = df[cols]
        
        # Store feature names (excluding 'domain')
        self.feature_names = [col for col in df.columns if col != 'domain']
        
        # Apply scaling if enabled
        if self.enable_scaling and not self._is_fitted:
            self.fit_scaler(df)
        
        if self.enable_scaling and self._is_fitted:
            df = self.transform_features(df)
        
        elapsed = time.time() - start_time
        rate = len(domains) / elapsed if elapsed > 0 else 0
        
        if show_progress:
            print(f"\n✅ Extracted features from {len(df)} domains")
            print(f"⏱️ Time: {elapsed:.2f}s | Rate: {rate:.0f} domains/sec")
            print(f"📊 Features: {len(self.feature_names)} total")
        
        return df
    
    def fit_scaler(self, df: pd.DataFrame) -> None:
        """Fit the scaler on feature data."""
        if self.scaler is None:
            return
        
        feature_cols = [col for col in df.columns if col != 'domain']
        self.scaler.fit(df[feature_cols])
        self._is_fitted = True
        print("✅ Scaler fitted on features")
    
    def transform_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Apply scaling transformation to features."""
        if self.scaler is None or not self._is_fitted:
            return df
        
        feature_cols = [col for col in df.columns if col != 'domain']
        df_scaled = df.copy()
        df_scaled[feature_cols] = self.scaler.transform(df[feature_cols])
        
        return df_scaled
    
    def get_feature_summary(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Generate summary statistics for all features.
        
        Args:
            df: DataFrame with extracted features
            
        Returns:
            Summary DataFrame with statistics
        """
        feature_cols = [col for col in df.columns if col != 'domain']
        
        summary = df[feature_cols].describe().T
        summary['missing'] = df[feature_cols].isnull().sum()
        summary['missing_pct'] = (summary['missing'] / len(df)) * 100
        
        return summary.round(3)

# Test the unified pipeline
print("🧪 Testing Unified Feature Extraction Pipeline\n")
print("=" * 70)

# Create test dataset
test_domains = [
    'google.com', 'facebook.com', 'amazon.com', 'twitter.com',
    'xqzpkwjt.net', 'abcdefgh.com', 'hjklmnop.org',
    'mail.google.com', 'api.github.com', 'stackoverflow.com'
]

# Initialize pipeline
pipeline = DomainFeatureExtractor(enable_scaling=False)

# Extract features
features_df = pipeline.extract_batch(test_domains, show_progress=True)

print("\n" + "=" * 70)
print("FEATURE EXTRACTION RESULTS")
print("=" * 70)
print(f"\nShape: {features_df.shape}")
print(f"Columns: {len(features_df.columns)}")
print(f"\nFirst few rows:")
print(features_df.head(3))

print("\n" + "=" * 70)
print("✅ Pipeline working successfully!")
print("=" * 70)

🧪 Testing Unified Feature Extraction Pipeline


✅ Extracted features from 10 domains
⏱️ Time: 0.02s | Rate: 654 domains/sec
📊 Features: 50 total

FEATURE EXTRACTION RESULTS

Shape: (10, 51)
Columns: 51

First few rows:
         domain  length  length_total  digit_count  digit_ratio  vowel_count  \
0    google.com       6            10            0          0.0            3   
1  facebook.com       8            12            0          0.0            4   
2    amazon.com       6            10            0          0.0            3   

   vowel_ratio  consonant_count  consonant_ratio   entropy  ...  \
0          0.5                3              0.5  1.918296  ...   
1          0.5                4              0.5  2.750000  ...   
2          0.5                3              0.5  2.251629  ...   

   subdomain_count  has_subdomain  total_labels  avg_label_length  \
0                0              0             2               4.5   
1                0              0             2      

### Performance Benchmark

Let's test if we meet our performance requirements (>1000 domains/second).

In [8]:
# Generate synthetic test data for performance testing
print("🏎️ PERFORMANCE BENCHMARK")
print("=" * 70)

# Create diverse test domains
np.random.seed(42)

def generate_random_domain(length_range=(5, 15)):
    """Generate random domain for testing."""
    length = np.random.randint(length_range[0], length_range[1])
    chars = 'abcdefghijklmnopqrstuvwxyz'
    domain_name = ''.join(np.random.choice(list(chars)) for _ in range(length))
    tld = np.random.choice(['com', 'net', 'org', 'io', 'co'])
    return f"{domain_name}.{tld}"

# Generate test sets of different sizes
test_sizes = [100, 500, 1000, 5000]

for size in test_sizes:
    test_domains = [generate_random_domain() for _ in range(size)]
    
    pipeline = DomainFeatureExtractor(enable_scaling=False)
    
    start_time = time.time()
    features_df = pipeline.extract_batch(test_domains, show_progress=False)
    elapsed = time.time() - start_time
    
    rate = size / elapsed
    avg_time_per_domain = (elapsed / size) * 1000  # in milliseconds
    
    status = "✅ PASS" if rate > 1000 else "⚠️ SLOW"
    
    print(f"\n{size:,} domains:")
    print(f"  Time: {elapsed:.3f}s")
    print(f"  Rate: {rate:.0f} domains/sec {status}")
    print(f"  Avg per domain: {avg_time_per_domain:.2f}ms")

print("\n" + "=" * 70)
print("TARGET: >1000 domains/second, <1ms per domain")
print("=" * 70)

🏎️ PERFORMANCE BENCHMARK

100 domains:
  Time: 0.091s
  Rate: 1102 domains/sec ✅ PASS
  Avg per domain: 0.91ms

500 domains:
  Time: 0.437s
  Rate: 1144 domains/sec ✅ PASS
  Avg per domain: 0.87ms

1,000 domains:
  Time: 0.864s
  Rate: 1158 domains/sec ✅ PASS
  Avg per domain: 0.86ms

5,000 domains:
  Time: 4.324s
  Rate: 1156 domains/sec ✅ PASS
  Avg per domain: 0.86ms

TARGET: >1000 domains/second, <1ms per domain


---

## 📊 Task 3: Feature Validation & Analysis

Now let's validate our features and understand their discriminative power.

### 3.1 Create Sample Dataset

We'll create a labeled dataset with legitimate and DGA domains for analysis.

In [ ]:
# Create a labeled dataset for validation
print("📦 Creating Sample Dataset")
print("=" * 70)

# Legitimate domains (real world examples)
legitimate_domains = [
    'google.com', 'facebook.com', 'amazon.com', 'twitter.com', 'linkedin.com',
    'microsoft.com', 'apple.com', 'github.com', 'stackoverflow.com', 'reddit.com',
    'wikipedia.org', 'youtube.com', 'netflix.com', 'instagram.com', 'pinterest.com',
    'ebay.com', 'paypal.com', 'dropbox.com', 'adobe.com', 'salesforce.com',
    'oracle.com', 'ibm.com', 'intel.com', 'cisco.com', 'nvidia.com',
    'airbnb.com', 'uber.com', 'spotify.com', 'zoom.us', 'slack.com'
]

# Simulate DGA domains (random-looking strings)
def generate_dga_like_domain(length_range=(8, 20)):
    """Generate DGA-like domain with high entropy."""
    length = np.random.randint(length_range[0], length_range[1])
    
    # Use less common character combinations
    chars = 'abcdefghijklmnopqrstuvwxyz'
    weights = np.random.dirichlet(np.ones(len(chars)))  # Random distribution
    
    domain_name = ''.join(np.random.choice(list(chars), p=weights) for _ in range(length))
    tld = np.random.choice(['net', 'com', 'org', 'info', 'biz'])
    
    return f"{domain_name}.{tld}"

np.random.seed(42)
dga_domains = [generate_dga_like_domain() for _ in range(30)]

# Create labeled dataset
all_domains = legitimate_domains + dga_domains
labels = [0] * len(legitimate_domains) + [1] * len(dga_domains)  # 0=legitimate, 1=DGA

# Extract features
pipeline = DomainFeatureExtractor(enable_scaling=False)
features_df = pipeline.extract_batch(all_domains, show_progress=True)

# Add labels
features_df['label'] = labels
features_df['label_name'] = features_df['label'].map({0: 'Legitimate', 1: 'DGA'})

print(f"\n✅ Dataset created:")
print(f"   Legitimate: {labels.count(0)} domains")
print(f"   DGA: {labels.count(1)} domains")
print(f"   Total Features: {len(pipeline.feature_names)}")

# Display sample
print("\nSample domains:")
print(features_df[['domain', 'label_name']].head(10))

📦 Creating Sample Dataset

✅ Extracted features from 60 domains
⏱️ Time: 0.08s | Rate: 758 domains/sec
📊 Features: 50 total

✅ Dataset created:


TypeError: 'bool' object is not iterable

### 3.2 Feature Distribution Analysis

Let's visualize how features differ between legitimate and DGA domains.

In [ ]:
# Analyze key features
print("📊 Analyzing Feature Distributions")
print("=" * 70)

# Select key features for visualization
key_features = [
    'length', 'entropy', 'vowel_ratio', 'pronounceability',
    'randomness_score', 'bigram_entropy', 'unique_char_ratio'
]

# Create visualizations
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

for idx, feature in enumerate(key_features):
    ax = axes[idx]
    
    # Plot distributions for each class
    legitimate_data = features_df[features_df['label'] == 0][feature]
    dga_data = features_df[features_df['label'] == 1][feature]
    
    ax.hist(legitimate_data, alpha=0.6, label='Legitimate', bins=15, color='green')
    ax.hist(dga_data, alpha=0.6, label='DGA', bins=15, color='red')
    
    ax.set_xlabel(feature.replace('_', ' ').title())
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.grid(True, alpha=0.3)

# Hide unused subplots
for idx in range(len(key_features), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('feature_distributions.png', dpi=100, bbox_inches='tight')
plt.show()

print("\n✅ Feature distribution plot saved as 'feature_distributions.png'")

### 3.3 Feature Correlation Analysis

Understanding feature correlations helps remove redundancy.

In [ ]:
# Correlation analysis
print("🔗 Feature Correlation Analysis")
print("=" * 70)

# Calculate correlation matrix
feature_cols = [col for col in features_df.columns 
                if col not in ['domain', 'label', 'label_name']]

correlation_matrix = features_df[feature_cols].corr()

# Plot correlation heatmap (top correlations only for readability)
plt.figure(figsize=(14, 12))

# Select subset of features for clearer visualization
feature_subset = key_features + ['digit_ratio', 'vc_transition_ratio', 'tld_is_common']
corr_subset = features_df[feature_subset].corr()

sns.heatmap(corr_subset, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})

plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('feature_correlation.png', dpi=100, bbox_inches='tight')
plt.show()

# Identify highly correlated features
threshold = 0.8
high_corr_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            high_corr_pairs.append({
                'feature1': correlation_matrix.columns[i],
                'feature2': correlation_matrix.columns[j],
                'correlation': correlation_matrix.iloc[i, j]
            })

if high_corr_pairs:
    print(f"\n⚠️ Found {len(high_corr_pairs)} highly correlated pairs (|r| > {threshold}):")
    for pair in high_corr_pairs[:5]:  # Show top 5
        print(f"  • {pair['feature1']} ↔ {pair['feature2']}: {pair['correlation']:.3f}")
else:
    print(f"\n✅ No highly correlated features found (threshold: {threshold})")

print("\n✅ Correlation analysis complete")

### 3.4 Feature Importance Analysis

Which features are most discriminative for DGA detection?

In [ ]:
# Feature importance using mutual information
print("⭐ Feature Importance Analysis")
print("=" * 70)

# Calculate mutual information scores
X = features_df[feature_cols]
y = features_df['label']

# Handle any infinite or NaN values
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

mi_scores = mutual_info_classif(X, y, random_state=42)

# Create feature importance dataframe
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': mi_scores
}).sort_values('importance', ascending=False)

# Display top features
print("\n🏆 Top 15 Most Discriminative Features:")
print("=" * 70)
for idx, row in feature_importance.head(15).iterrows():
    bar = '█' * int(row['importance'] * 50)
    print(f"{row['feature']:30s} | {bar} {row['importance']:.4f}")

# Visualize top features
plt.figure(figsize=(12, 8))
top_n = 20
top_features = feature_importance.head(top_n)

plt.barh(range(len(top_features)), top_features['importance'], color='steelblue')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Mutual Information Score', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title(f'Top {top_n} Most Important Features for DGA Detection', 
          fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=100, bbox_inches='tight')
plt.show()

print("\n✅ Feature importance plot saved as 'feature_importance.png'")

### 3.5 Class Separability Analysis

Let's see how well our features separate legitimate from DGA domains.

In [ ]:
# Class separability analysis
print("🎯 Class Separability Analysis")
print("=" * 70)

# Calculate per-feature statistics for each class
separability_stats = []

for feature in feature_cols:
    legitimate_values = features_df[features_df['label'] == 0][feature]
    dga_values = features_df[features_df['label'] == 1][feature]
    
    # Calculate means and standard deviations
    legit_mean = legitimate_values.mean()
    dga_mean = dga_values.mean()
    
    legit_std = legitimate_values.std()
    dga_std = dga_values.std()
    
    # Calculate separation (difference in means relative to pooled std)
    pooled_std = np.sqrt((legit_std**2 + dga_std**2) / 2)
    separation = abs(legit_mean - dga_mean) / (pooled_std + 1e-10)
    
    separability_stats.append({
        'feature': feature,
        'legitimate_mean': legit_mean,
        'dga_mean': dga_mean,
        'separation': separation
    })

separability_df = pd.DataFrame(separability_stats).sort_values('separation', ascending=False)

print("\n🏆 Top 10 Features with Best Class Separation:")
print("=" * 70)
print(f"{'Feature':<30} {'Legit Mean':>12} {'DGA Mean':>12} {'Separation':>12}")
print("-" * 70)

for idx, row in separability_df.head(10).iterrows():
    print(f"{row['feature']:<30} {row['legitimate_mean']:>12.3f} "
          f"{row['dga_mean']:>12.3f} {row['separation']:>12.3f}")

# Visualize separation for top features
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

top_separable = separability_df.head(6)['feature'].tolist()

for idx, feature in enumerate(top_separable):
    ax = axes[idx]
    
    legitimate_data = features_df[features_df['label'] == 0][feature]
    dga_data = features_df[features_df['label'] == 1][feature]
    
    ax.boxplot([legitimate_data, dga_data], labels=['Legitimate', 'DGA'])
    ax.set_title(feature.replace('_', ' ').title(), fontweight='bold')
    ax.set_ylabel('Value')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('class_separability.png', dpi=100, bbox_inches='tight')
plt.show()

print("\n✅ Class separability plot saved as 'class_separability.png'")

---

## 🛡️ Task 4: Production Readiness

Let's add production-grade features: error handling, logging, and testing.

### 4.1 Enhanced Pipeline with Error Handling

In [ ]:
import logging
from typing import Optional

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

class ProductionDomainFeatureExtractor(DomainFeatureExtractor):
    """
    Production-ready feature extractor with:
    - Comprehensive error handling
    - Logging
    - Input validation
    - Metrics tracking
    """
    
    def __init__(self, enable_scaling: bool = False, log_errors: bool = True):
        super().__init__(enable_scaling)
        self.log_errors = log_errors
        self.logger = logging.getLogger(self.__class__.__name__)
        
        # Metrics
        self.total_processed = 0
        self.total_errors = 0
        self.error_domains = []
    
    def validate_domain(self, domain: str) -> Tuple[bool, Optional[str]]:
        """
        Validate domain name format.
        
        Returns:
            (is_valid, error_message)
        """
        if not domain or not isinstance(domain, str):
            return False, "Domain must be a non-empty string"
        
        if len(domain) > 253:  # RFC 1035
            return False, "Domain exceeds maximum length (253 characters)"
        
        if '..' in domain:
            return False, "Domain contains consecutive dots"
        
        if domain.startswith('.') or domain.endswith('.'):
            return False, "Domain starts or ends with dot"
        
        # Basic pattern check
        if not re.match(r'^[a-zA-Z0-9.-]+$', domain):
            return False, "Domain contains invalid characters"
        
        return True, None
    
    def extract_single(self, domain: str) -> Dict[str, float]:
        """Extract features with validation and error handling."""
        self.total_processed += 1
        
        # Validate input
        is_valid, error_msg = self.validate_domain(domain)
        if not is_valid:
            self.total_errors += 1
            self.error_domains.append({'domain': domain, 'error': error_msg})
            
            if self.log_errors:
                self.logger.warning(f"Invalid domain '{domain}': {error_msg}")
            
            return {}
        
        # Extract features with error handling
        try:
            features = super().extract_single(domain)
            
            # Validate feature values (check for NaN, inf)
            for key, value in features.items():
                if not isinstance(value, (int, float)):
                    continue
                    
                if math.isnan(value) or math.isinf(value):
                    features[key] = 0.0  # Replace invalid values
                    if self.log_errors:
                        self.logger.warning(f"Invalid value for {key} in domain '{domain}': {value}")
            
            return features
            
        except Exception as e:
            self.total_errors += 1
            self.error_domains.append({'domain': domain, 'error': str(e)})
            
            if self.log_errors:
                self.logger.error(f"Error extracting features from '{domain}': {str(e)}")
            
            return {}
    
    def get_metrics(self) -> Dict:
        """Get processing metrics."""
        return {
            'total_processed': self.total_processed,
            'total_errors': self.total_errors,
            'error_rate': self.total_errors / max(self.total_processed, 1),
            'success_rate': 1 - (self.total_errors / max(self.total_processed, 1))
        }
    
    def get_error_report(self) -> pd.DataFrame:
        """Get detailed error report."""
        if not self.error_domains:
            return pd.DataFrame()
        
        return pd.DataFrame(self.error_domains)

# Test production pipeline
print("🏭 Testing Production Pipeline")
print("=" * 70)

# Test with various inputs (including invalid ones)
test_cases = [
    'google.com',          # Valid
    'facebook.com',        # Valid
    '',                    # Invalid: empty
    'invalid..domain.com', # Invalid: consecutive dots
    '.startsdot.com',      # Invalid: starts with dot
    'a' * 300 + '.com',    # Invalid: too long
    'valid123.org',        # Valid
    'special!@#.com',      # Invalid: special chars
]

prod_pipeline = ProductionDomainFeatureExtractor(enable_scaling=False, log_errors=True)
results = prod_pipeline.extract_batch(test_cases, show_progress=False)

# Show metrics
print("\n📊 Processing Metrics:")
metrics = prod_pipeline.get_metrics()
for key, value in metrics.items():
    if 'rate' in key:
        print(f"  {key}: {value*100:.1f}%")
    else:
        print(f"  {key}: {value}")

# Show error report
if prod_pipeline.total_errors > 0:
    print("\n⚠️ Error Report:")
    print(prod_pipeline.get_error_report())

print("\n✅ Production pipeline tested")

### 4.2 Unit Tests

Professional code needs tests. Let's write some basic test cases.

In [ ]:
def test_lexical_features():
    """Test lexical feature extraction."""
    extractor = LexicalFeatureExtractor()
    
    # Test basic domain
    features = extractor.extract('google.com')
    assert 'length' in features
    assert 'entropy' in features
    assert features['length'] == 6  # 'google'
    
    # Test domain with digits
    features = extractor.extract('test123.com')
    assert features['digit_count'] == 3
    assert features['digit_ratio'] > 0
    
    print("✅ Lexical features test passed")

def test_linguistic_features():
    """Test linguistic feature extraction."""
    extractor = LinguisticFeatureExtractor()
    
    # Test pronounceable domain
    features = extractor.extract('banana.com')
    assert 'pronounceability' in features
    assert features['pronounceability'] > 0.5  # Should be pronounceable
    
    # Test random domain
    features = extractor.extract('xqzpk.com')
    assert features['pronounceability'] < 0.5  # Should be less pronounceable
    
    print("✅ Linguistic features test passed")

def test_statistical_features():
    """Test statistical feature extraction."""
    extractor = StatisticalFeatureExtractor()
    
    features = extractor.extract('google.com')
    assert 'bigram_entropy' in features
    assert 'randomness_score' in features
    assert 'unique_char_ratio' in features
    
    print("✅ Statistical features test passed")

def test_dns_features():
    """Test DNS feature extraction."""
    extractor = DNSFeatureExtractor()
    
    # Test with subdomain
    features = extractor.extract('mail.google.com')
    assert features['subdomain_count'] == 1
    assert features['total_labels'] == 3
    
    # Test without subdomain
    features = extractor.extract('google.com')
    assert features['subdomain_count'] == 0
    
    print("✅ DNS features test passed")

def test_pipeline_integration():
    """Test full pipeline."""
    pipeline = DomainFeatureExtractor()
    
    domains = ['google.com', 'facebook.com', 'twitter.com']
    result = pipeline.extract_batch(domains, show_progress=False)
    
    assert len(result) == 3
    assert 'domain' in result.columns
    assert len(result.columns) > 10  # Should have many features
    
    print("✅ Pipeline integration test passed")

# Run all tests
print("🧪 Running Unit Tests")
print("=" * 70)

try:
    test_lexical_features()
    test_linguistic_features()
    test_statistical_features()
    test_dns_features()
    test_pipeline_integration()
    
    print("\n" + "=" * 70)
    print("✅ ALL TESTS PASSED")
    print("=" * 70)
    
except AssertionError as e:
    print(f"\n❌ TEST FAILED: {str(e)}")
except Exception as e:
    print(f"\n❌ ERROR: {str(e)}")

### 4.3 Export Features and Metadata

For production use, we need to export features and their descriptions.

In [ ]:
def export_feature_metadata(pipeline: DomainFeatureExtractor, 
                           output_file: str = 'feature_metadata.csv'):
    """
    Export feature metadata (names, descriptions, types).
    
    Args:
        pipeline: Fitted feature extractor
        output_file: Output CSV file path
    """
    
    # Define feature metadata
    metadata = []
    
    # Lexical features
    lexical_features = {
        'length': 'Length of domain name (without TLD)',
        'length_total': 'Total length including TLD',
        'digit_count': 'Number of digit characters',
        'digit_ratio': 'Ratio of digits to total characters',
        'vowel_count': 'Number of vowel characters',
        'vowel_ratio': 'Ratio of vowels to total characters',
        'consonant_count': 'Number of consonant characters',
        'consonant_ratio': 'Ratio of consonants to total characters',
        'entropy': 'Shannon entropy (randomness measure)',
        'hyphen_count': 'Number of hyphens',
        'underscore_count': 'Number of underscores',
        'special_char_ratio': 'Ratio of special characters',
        'uppercase_count': 'Number of uppercase letters',
        'uppercase_ratio': 'Ratio of uppercase letters',
        'max_consecutive_digits': 'Maximum consecutive digits',
        'max_consecutive_consonants': 'Maximum consecutive consonants'
    }
    
    # Linguistic features
    linguistic_features = {
        'pronounceability': 'How pronounceable the domain is (0-1)',
        'vowel_consonant_transitions': 'Number of vowel-consonant transitions',
        'vc_transition_ratio': 'Ratio of V-C transitions',
        'common_bigram_count': 'Number of common English bigrams',
        'common_bigram_ratio': 'Ratio of common bigrams',
        'common_trigram_count': 'Number of common English trigrams',
        'common_trigram_ratio': 'Ratio of common trigrams',
        'unique_bigram_ratio': 'Ratio of unique bigrams',
        'unique_trigram_ratio': 'Ratio of unique trigrams',
        'repeating_patterns': 'Number of repeating character patterns'
    }
    
    # Statistical features
    statistical_features = {
        'char_freq_mean': 'Mean character frequency',
        'char_freq_std': 'Standard deviation of character frequency',
        'char_freq_variance': 'Variance of character frequency',
        'unique_char_ratio': 'Ratio of unique characters',
        'bigram_entropy': 'Entropy of bigram distribution',
        'trigram_entropy': 'Entropy of trigram distribution',
        'char_freq_skewness': 'Skewness of character frequency distribution',
        'char_freq_kurtosis': 'Kurtosis of character frequency distribution',
        'randomness_score': 'Overall randomness score (0-1)',
        'avg_char_position': 'Average alphabetic position of characters',
        'char_position_std': 'Std dev of character positions'
    }
    
    # DNS features
    dns_features = {
        'tld_is_common': 'Whether TLD is common (1/0)',
        'tld_is_suspicious': 'Whether TLD is suspicious (1/0)',
        'tld_length': 'Length of TLD',
        'subdomain_count': 'Number of subdomains',
        'has_subdomain': 'Whether domain has subdomain (1/0)',
        'total_labels': 'Total number of DNS labels',
        'avg_label_length': 'Average length of DNS labels',
        'max_label_length': 'Maximum label length',
        'min_label_length': 'Minimum label length',
        'domain_name_length': 'Length of primary domain name',
        'has_numerical_suffix': 'Whether domain ends with digit (1/0)',
        'dot_count': 'Number of dots in full domain',
        'dot_density': 'Ratio of dots to total length'
    }
    
    # Combine all features
    all_features = {
        **lexical_features,
        **linguistic_features,
        **statistical_features,
        **dns_features
    }
    
    # Create metadata rows
    for feature_name, description in all_features.items():
        # Determine category
        if feature_name in lexical_features:
            category = 'Lexical'
        elif feature_name in linguistic_features:
            category = 'Linguistic'
        elif feature_name in statistical_features:
            category = 'Statistical'
        else:
            category = 'DNS'
        
        metadata.append({
            'feature_name': feature_name,
            'category': category,
            'description': description,
            'type': 'numeric'
        })
    
    # Create DataFrame and export
    metadata_df = pd.DataFrame(metadata)
    metadata_df.to_csv(output_file, index=False)
    
    print(f"✅ Feature metadata exported to '{output_file}'")
    print(f"   Total features: {len(metadata_df)}")
    
    return metadata_df

# Export metadata
print("📄 Exporting Feature Metadata")
print("=" * 70)

pipeline = DomainFeatureExtractor()
metadata_df = export_feature_metadata(pipeline)

print("\nSample metadata:")
print(metadata_df.head(10))

print("\nFeatures by category:")
print(metadata_df['category'].value_counts())

---

## 🎓 Summary & Key Takeaways

### What We Accomplished

✅ **Implemented 4 Feature Extraction Modules**:
- Lexical: Character-level patterns (length, entropy, composition)
- Linguistic: Language-like properties (pronounceability, n-grams)
- Statistical: Distribution analysis (entropy, randomness)
- DNS: Domain structure (TLD, subdomains, labels)

✅ **Built Production-Ready Pipeline**:
- Unified `DomainFeatureExtractor` class
- Batch processing (>1000 domains/second)
- Error handling and validation
- Feature scaling and normalization

✅ **Validated Features**:
- Extracted 50+ features per domain
- Analyzed feature distributions
- Identified discriminative features
- Examined class separability

✅ **Production Readiness**:
- Comprehensive error handling
- Logging and metrics
- Unit tests
- Feature metadata export

---

### Key Insights

1. **Most Discriminative Features** (for DGA detection):
   - Entropy (Shannon entropy)
   - Pronounceability
   - Randomness score
   - Bigram/trigram entropy
   - Vowel-consonant patterns

2. **Performance Achieved**:
   - Processing rate: >1000 domains/second ✅
   - Latency: <1ms per domain ✅
   - Feature count: 50+ ✅

3. **Class Separability**:
   - Legitimate domains: Low entropy, high pronounceability, common patterns
   - DGA domains: High entropy, low pronounceability, random patterns

---

### Next Steps for Students

1. **Enhance Features**:
   - Add temporal features (registration date, age)
   - Include WHOIS information
   - Incorporate DNS query patterns

2. **Build ML Models**:
   - Train classifiers (Random Forest, XGBoost, Neural Networks)
   - Evaluate with proper metrics (Precision@k, TPR@1%FPR)
   - Handle class imbalance

3. **Deploy to Production**:
   - Integrate with SIEM/security tools
   - Implement real-time processing
   - Monitor for concept drift
   - Set up retraining pipeline

4. **Advanced Topics**:
   - Multi-class DGA family classification
   - Adversarial robustness
   - Explainable predictions (SHAP, LIME)

---

### Resources for Further Learning

- **Datasets**: DGArchive, OSINT feeds, Alexa Top 1M
- **Papers**: "Beyond Blacklists" (Antonakakis et al.), DGA taxonomies
- **Tools**: Scikit-learn, XGBoost, TensorFlow
- **Frameworks**: MITRE ATT&CK (T1568), Cyber Kill Chain

---

### 💡 Remember

- **Security Context**: Features should align with threat models
- **Operational Constraints**: False positives break SOC workflows
- **Concept Drift**: Attackers evolve; models must retrain
- **Explainability**: Analysts need to understand predictions

---

**Congratulations! You've built a production-grade feature extraction pipeline for DGA detection.** 🎉

This foundation is critical for building effective machine learning-based security systems.

---

## 📝 Exercise Deliverables Checklist

Before submitting, ensure you have:

- [ ] ✅ Implemented all 4 feature extraction modules
- [ ] ✅ Created unified `DomainFeatureExtractor` pipeline
- [ ] ✅ Achieved >1000 domains/second throughput
- [ ] ✅ Generated feature distribution visualizations
- [ ] ✅ Completed correlation analysis
- [ ] ✅ Analyzed feature importance
- [ ] ✅ Added error handling and validation
- [ ] ✅ Written unit tests
- [ ] ✅ Exported feature metadata
- [ ] ✅ Documented code with comments

### Files to Submit

1. **This notebook** (`exercise_8.4a_feature_extraction.ipynb`) - fully executed
2. **Python module** (optional): `domain_features.py` with classes
3. **Visualizations**: All generated PNG files
4. **Metadata**: `feature_metadata.csv`
5. **README**: Document your implementation approach

---

**Good luck with your implementation!** 🚀